In [6]:
!pip install -q datasets
!pip install opensearch-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 8.9 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

# 选择要下载的数据集名称，可以在 Hugging Face 数据集网站上查找
dataset_name = "squad_v2"

# 下载数据集
dataset = load_dataset(dataset_name)

# 打印数据集信息
print(dataset)

# 访问数据集中的示例
sample = dataset["train"][0]
print(sample)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "C

In [14]:
import boto3
import requests
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers

service = 'es'
session = boto3.Session()
credentials = session.get_credentials()
region = session.region_name
auth = AWSV4SignerAuth(credentials, region)
aos_endpoint='vpc-domain66ac69e0-ijsmtgwnje5s-oa63og27tmacx2fstb5hpstta4.us-west-2.es.amazonaws.com'

client = OpenSearch(
    hosts = [{'host': aos_endpoint, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    timeout = 60, # 默认超时时间是10 秒，
    max_retries=5, # 重试次数
    retry_on_timeout=True
)

index_name = "your-index-name"
index_mapping = {
    "mappings": {
        "properties": {
            "field1": {"type": "text"},
            "field2": {"type": "integer"},
            # 添加其他字段和其数据类型
        }
    }
}

# 创建索引
response = client.indices.delete(index=index_name)
# response = client.indices.create(index=index_name, body=index_mapping)
print(response)


{'acknowledged': True}


In [ ]:
import json

request_body = {
  "parameters": {
    "texts": ["Hello word", "Hi Altman"]
  }
}

# Execute the _predict request
response = client.transport.perform_request(
    method="POST",
    url=f"/_plugins/_ml/models/4EtIC4wB0S9ucTLoWVtt/_predict",
    body=json.dumps(request_body)
)

request_body = {
  "description": "neural sparse encoding pipeline",
  "processors" : [
    {
      "sparse_encoding": {
        "model_id": "<nerual_sparse_model_id>",
        "field_map": {
           "content": "sparse_embedding"
        }
      }
    },
    {
      "text_embedding": {
        "model_id": "<cohere_ingest_model_id>",
        "field_map": {
          "doc": "embedding"
        }
      }
    }
  ]
}

response = client.transport.perform_request(
    method="PUT",
    url=f"/_ingest/pipeline/dense-sparse-pipeline",
    body=json.dumps(request_body)
)

PUT /_ingest/pipeline/dense-sparse-pipeline
{
  "description": "neural sparse encoding pipeline",
  "processors" : [
    {
      "sparse_encoding": {
        "model_id": "<nerual_sparse_model_id>",
        "field_map": {
           "content": "sparse_embedding"
        }
      }
    },
    {
      "text_embedding": {
        "model_id": "<cohere_ingest_model_id>",
        "field_map": {
          "doc": "embedding"
        }
      }
    }
  ]
}

GET /_plugins/_ml/models/

print(response)

In [17]:
response = client.transport.perform_request(
    method="GET",
    url="/_ml/trained_models"
)

print(response)

AuthenticationException: AuthenticationException(401, '{"Message":"Your request: \'/_ml/trained_models\' is not allowed."}')

In [8]:
help(client)

Help on OpenSearch in module opensearchpy.client object:

class OpenSearch(opensearchpy.client.client.Client)
 |  OpenSearch(hosts: Any = None, transport_class: Type[opensearchpy.transport.Transport] = <class 'opensearchpy.transport.Transport'>, **kwargs: Any) -> None
 |  
 |  OpenSearch client. Provides a straightforward mapping from
 |  Python to OpenSearch REST endpoints.
 |  
 |  The instance has attributes ``cat``, ``cluster``, ``indices``, ``ingest``,
 |  ``nodes``, ``snapshot`` and ``tasks`` that provide access to instances of
 |  :class:`~opensearchpy.client.CatClient`,
 |  :class:`~opensearchpy.client.ClusterClient`,
 |  :class:`~opensearchpy.client.IndicesClient`,
 |  :class:`~opensearchpy.client.IngestClient`,
 |  :class:`~opensearchpy.client.NodesClient`,
 |  :class:`~opensearchpy.client.SnapshotClient` and
 |  :class:`~opensearchpy.client.TasksClient` respectively. This is the
 |  preferred (and only supported) way to get access to those classes and their
 |  methods.
 |  